# NTTRU Estimation
Estimate the drop of bit security caused by multiple information leaked from power side channel.

In [1]:
load("../framework/instance_gen.sage")

In [2]:
## NTRU instance initilizaiton


n = 768
m = n
q = 7681

#number of one in f and g In NTTRU , the distribution of -1,0,1 is 5/16, 6/16,6/16
D_e = {-1: 5/16, 0:6/16, 1:5/16}
D_s = D_e

bit_security_constant = 0.292
A, b, dbdd = initialize_from_LWE_instance(DBDD_predict, n, q, m, D_e,D_s)

      Build DBDD from LWE      
 n=768 	 m=768 	 q=7681 


In [3]:
s_key = []

for i in range(n):
    v0 = [0 for i in range(m + n)]
    v0[i]=1
    s_key.append(dbdd.leak(v0))

In [4]:
## NTT initialization
R = IntegerModRing(q)
V = VectorSpace(R,n)

import numpy as np

NTT_matrix = []

def add (x,y) : return x +y

#zetas in the last layer of InvNTT 
zetas=[-7661,7661,-6212,6212,-6464,6464,7586,-7586,4822,-4822,1618,-1618,3845,-3845,-3702,3702,4277,-4277,1913,-1913,3326,-3326,-6874,6874,-3446,3446,5741,-5741,-1534,1534,-2834,2834,1081,-1081,-867,867,5483,-5483,626,-626,-1293,1293,3730,-3730,-1485,1485,-7300,7300,1894,-1894,6233,-6233,803,-803,-5156,5156,-7289,7289,-5004,5004,-726,726,5819,-5819,-245,245,-713,713,-5307,5307,3084,-3084,-1462,1462,7063,-7063,-6776,6776,3104,-3104,3294,-3294,-6152,6152,1502,-1502,-4125,4125,-32,32,722,-722,7270,-7270,-7529,7529,4040,-4040,-2821,2821,42,-42,-3828,3828,6238,-6238,4234,-4234,-6772,6772,-2747,2747,-3999,3999,-5305,5305,-4076,4076,-5968,5968,2879,-2879,-149,149,5053,-5053,-4074,4074,-2977,2977,1400,-1400,6650,-6650,699,-699,1955,-1955,7257,-7257,5667,-5667,-7366,7366,4348,-4348,7512,-7512,-2723,2723,2390,-2390,-2458,2458,4572,-4572,6355,-6355,-7527,7527,-758,758,6541,-6541,2266,-2266,7441,-7441,-7627,7627,-6019,6019,-3627,3627,-4097,4097,1507,-1507,2003,-2003,-87,87,2443,-2443,-3046,3046,4397,-4397,-237,237,2947,-2947,3824,-3824,5893,-5893,-812,812,-2802,2802,-2826,2826,5194,-5194,-2212,2212,1902,-1902,504,-504,-7531,7531,4553,-4553,-2394,2394,3227,-3227,-2240,2240,4722,-4722,-5727,5727,-2236,2236,-1397,1397,6806,-6806,-4741,4741,-4502,4502,6524,-6524,265,-265,2182,-2182,-5019,5019,-3414,3414,2986,-2986,6558,-6558,2749,-2749,-5857,5857,983,-983,384,-384]

def modular_exponentiation(base, exponent, modulus):
    result = 1
    base = base % modulus  
    while exponent > 0:
        if (exponent % 2) == 1:  
            result = (result * base) % modulus
        exponent //= 2  
        base = (base * base) % modulus 
    return result


for x in range(n/3):
    NTT_matrix.append(V(reduce(add, [[modular_exponentiation(zetas[i],x,7681),0,0] for i in range(n/3)])))
    NTT_matrix.append(V(reduce(add, [[0,modular_exponentiation(zetas[i],x,7681),0] for i in range(n/3)])))
    NTT_matrix.append(V(reduce(add, [[0,0,modular_exponentiation(zetas[i],x,7681)] for i in range(n/3)])))

NTT_matrix = matrix(NTT_matrix)

In [5]:
#Get s_hat
s_hat_list = V(s_key)*NTT_matrix

In [6]:
v_list = []

# Get nums_z

nums_z = []*256

for i in range(256):
    term1 = s_hat_list[3 * i]          # f[3*i]
    term2 = s_hat_list[3 * i + 1]      # f[3*i+1]
    term3 = s_hat_list[3 * i + 2]      # f[3*i+2]

    # z_i=R(-term2-term3)/R(term1)
    z_i = 0 if R(term1) == 0 else R(-term2 - term3) / R(term1)
    z_i = R(z_i)
    
    nums_z.append(z_i)

In [7]:
# Hints are stored in v_list
for i in range(256):
    v=nums_z[i]*NTT_matrix.column(3*i)+NTT_matrix.column(3*i+1)+NTT_matrix.column(3*i+2)
    v=list(v)
    v_prime = [int(i) for i in list(v)] + [0]*768
    v_list.append(vec(v_prime))
    

In [8]:
for v_0 in v_list:
    a=dbdd.leak(v_0)%7681
    #If the hint is not valid, skip it
    if(a!=0):  
        continue
    dbdd.integrate_modular_hint(v_0,0,7681,True)

 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00327470, β=527.88 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00327850, β=527.13 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00328170, β=526.38 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00328490, β=525.63 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00328809, β=524.88 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00329189, β=524.13 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00329509, β=523.39 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00329828, β=522.65 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00330148, β=521.91 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00330528, β=521.17 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00330847, β=520.43 
 integrate modular hi

 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00348655, β=483.50 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00348969, β=482.84 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00349349, β=482.18 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00349664, β=481.53 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00349978, β=480.87 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00350358, β=480.22 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00350672, β=479.57 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00350986, β=478.92 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00351366, β=478.27 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00351680, β=477.62 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00351994, β=476.98 
 integrate modular hi

 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00369944, β=444.56 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00370253, β=443.98 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00370634, β=443.40 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00370943, β=442.82 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00371323, β=442.25 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00371631, β=441.67 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00372012, β=441.10 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00372320, β=440.53 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00372629, β=439.95 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00373009, β=439.38 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00373317, β=438.82 
 integrate modular hi

 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00391387, β=410.18 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00391691, β=409.66 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00392071, β=409.15 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00392374, β=408.64 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00392755, β=408.13 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00393058, β=407.62 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00393438, β=407.11 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00393741, β=406.61 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00394121, β=406.10 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00394424, β=405.60 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00394804, β=405.09 
 integrate modular hi

 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00412874, β=379.65 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00413255, β=379.19 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00413552, β=378.74 
 integrate modular hint   (smooth)   	 Worthy hint !   dim=1537, δ=1.00413932, β=378.28 


In [9]:
378.28*0.292

110.457760000000